In [4]:
!pip install ckanapi
from ckanapi import *
import csv
import requests

api = RemoteCKAN('https://dadosabertos.capes.gov.br')
datasets = api.action.package_list()


  Using cached ckanapi-4.8-py3-none-any.whl (46 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
  Using cached simplejson-3.19.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (137 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)


In [5]:
#Função para filtrar a lista de datasets, colocando os que se adequam ao filtro
def filterCatch(data, filter):
  return [str for str in data if
          any(sub in str for sub in filter)]

#Função para filtrar a lista de datasets, retirando os que se adequam ao filtro
def filterMiss(data, filter):
  return [str for str in data if
          not any(sub in str for sub in filter)]

In [6]:
filters=["2021-a-2024"]
database=filterCatch(datasets,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-artistica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu

In [7]:
filters=["artistica"]
database=filterMiss(database,filters)
print(*database,sep="\n")

2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil
2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao
2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu
2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil
2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil
2021-a-2024-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil


In [8]:
import logging
import time

import os

def download_dataset(database, caminho, max_attempts=3):
    for dataset in database:
        try:
            archive = api.action.package_show(id=dataset)
            link_archive = archive['resources'][0]['url']
            name_archive = caminho + dataset + '.csv'

            # Verificar se o arquivo já existe

            for attempt in range(max_attempts):
                try:
                    print(f"Tentativa {attempt + 1}/{max_attempts}")
                    !wget -O {name_archive} --retry-connrefused {link_archive}
                    break  # Download bem-sucedido

                except ConnectionError as e:
                    logging.error(f"Erro de conexão ao baixar dataset {dataset} (tentativa {attempt + 1}/{max_attempts}): {e}")
                    time.sleep(60)  # Aguardar 1 minuto antes de tentar novamente

                    # Tentar continuar com -c em caso de erro de conexão
                    !wget -c {name_archive} {link_archive}

                except Exception as e:
                    logging.error(f"Erro inesperado ao baixar dataset {dataset}: {e}")

        except Exception as e:
            logging.error(f"Erro ao obter informações do dataset {dataset}: {e}")

In [11]:
download_dataset(database,'csv/')

Tentativa 1/3
--2024-08-02 21:02:45--  https://dadosabertos.capes.gov.br/dataset/d7ee8c2b-253b-4619-884c-18f376b619ac/resource/bb07b752-132f-4d61-ae99-707b40a8743a/download/br-capes-colsucup-prod-autor-2021a2024-2023-10-31-artistica-cenica.csv
Resolving dadosabertos.capes.gov.br (dadosabertos.capes.gov.br)... 200.130.18.196, 200.130.18.195, 2001:12f0:b27:1004::245, ...
Connecting to dadosabertos.capes.gov.br (dadosabertos.capes.gov.br)|200.130.18.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 756372 (739K) [text/csv]
Saving to: ‘csv/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv’

csv/2021-a-2024-aut 100%[===================>] 738.64K   211KB/s    in 3.7s    

2024-08-02 21:02:49 (199 KB/s) - ‘csv/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv’ saved [756372/756372]

Tentativa 1/3
--2024-08-02 21:02:49--  https://dadosabertos.capes.gov.br/dataset/dbb9